In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# !pip install transformers[torch]

In [1]:
print("hello world")

hello world


In [2]:
# import itertools
# import logging
# from typing import Optional, Dict, Union

from nltk import sent_tokenize

import torch
from transformers import(
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
     T5Tokenizer, BartTokenizer,T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
)





2024-04-13 15:06:39.738090: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-13 15:06:39.738220: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-13 15:06:39.893844: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from datasets import load_dataset, load_metric

In [4]:
dataset = load_dataset("squad")


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [5]:

def preprocess_function(examples):
   
    inputs = ["context: " + doc for doc in examples['context']]
    targets = [question for question in examples['question']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")


    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenizer = T5Tokenizer.from_pretrained('t5-base')
tokenized_datasets = dataset.map(preprocess_function, batched=True)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [6]:
model = T5ForConditionalGeneration.from_pretrained('t5-base')


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
train_dataset=tokenized_datasets['train']

In [39]:
train_dataset

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 87599
})

In [9]:
import pandas as pd
train_dataset = tokenized_datasets['train']
train_df = train_dataset.to_pandas()



In [41]:
val_dataset=tokenized_datasets['validation']

In [42]:
val_df = train_dataset.to_pandas()

In [43]:
val_df.head()

,id,title,context,question,answers,input_ids,attention_mask,labels
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ...","[2625, 10, 30797, 120, 6, 8, 496, 65, 3, 9, 65...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[304, 4068, 410, 8, 16823, 3790, 3, 18280, 238..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe...","[2625, 10, 30797, 120, 6, 8, 496, 65, 3, 9, 65...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[363, 19, 16, 851, 13, 8, 7711, 3, 17084, 5140..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'...","[2625, 10, 30797, 120, 6, 8, 496, 65, 3, 9, 65...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[37, 23711, 2617, 13, 8, 3, 24756, 842, 44, 77..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...,"[2625, 10, 30797, 120, 6, 8, 496, 65, 3, 9, 65...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[363, 19, 8, 8554, 17, 235, 44, 7711, 3, 17084..."
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...,"[2625, 10, 30797, 120, 6, 8, 496, 65, 3, 9, 65...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[363, 2561, 7, 30, 420, 13, 8, 5140, 5450, 44,..."


In [10]:
train_df.head()

,id,title,context,question,answers,input_ids,attention_mask,labels
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ...","[2625, 10, 30797, 120, 6, 8, 496, 65, 3, 9, 65...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[304, 4068, 410, 8, 16823, 3790, 3, 18280, 238..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe...","[2625, 10, 30797, 120, 6, 8, 496, 65, 3, 9, 65...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[363, 19, 16, 851, 13, 8, 7711, 3, 17084, 5140..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'...","[2625, 10, 30797, 120, 6, 8, 496, 65, 3, 9, 65...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[37, 23711, 2617, 13, 8, 3, 24756, 842, 44, 77..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...,"[2625, 10, 30797, 120, 6, 8, 496, 65, 3, 9, 65...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[363, 19, 8, 8554, 17, 235, 44, 7711, 3, 17084..."
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...,"[2625, 10, 30797, 120, 6, 8, 496, 65, 3, 9, 65...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[363, 2561, 7, 30, 420, 13, 8, 5140, 5450, 44,..."


In [13]:
!pip install transformers[torch]

In [37]:
training_args = TrainingArguments(
     fp16=True,
    output_dir="./e2e",          # The output directory
    evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    disable_tqdm=False,
    push_to_hub=False,
    gradient_accumulation_steps = 2,
     

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer
)


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
torch.cuda.empty_cache() 

In [15]:
trainer.train()


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
0,0.210400,0.193034
2,0.196400,0.189344


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=8211, training_loss=0.24606679206265555, metrics={'train_runtime': 18175.2131, 'train_samples_per_second': 14.459, 'train_steps_per_second': 0.452, 'total_flos': 1.0602528843105792e+17, 'train_loss': 0.24606679206265555, 'epoch': 3.0})

In [16]:
model_path = "/kaggle/working"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


('/kaggle/working/tokenizer_config.json',
 '/kaggle/working/special_tokens_map.json',
 '/kaggle/working/spiece.model',
 '/kaggle/working/added_tokens.json')

In [17]:
eval_results = trainer.evaluate()
print(eval_results)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.1893439143896103, 'eval_runtime': 231.953, 'eval_samples_per_second': 45.57, 'eval_steps_per_second': 2.85, 'epoch': 3.0}


In [25]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Replace 'path_to_trained_model' with the path where your trained model is saved
model_path = '/kaggle/working'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [51]:
def generate_question(text, model=model, tokenizer=tokenizer, max_length=64):
    # Encode the text input ensuring padding and truncation
    input_ids = tokenizer.encode("context: " + text, return_tensors="pt", max_length=512, padding="max_length", truncation=True).to(model.device)

    # Generate the output sequence
    output_ids = model.generate(input_ids, max_length=max_length, num_beams=4, early_stopping=True)

    # Decode the output to text
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question


In [27]:
# Example text
text = "Albert Einstein was a German-born theoretical physicist who developed the theory of relativity, one of the two pillars of modern physics. His work is also known for its influence on the philosophy of science."

# Generate a question
question = generate_question(text, model, tokenizer)
print("Generated Question:", question)


Generated Question: What is Einstein's theory of relativity?


In [32]:
text="Mahima Loves Python.Python is Great.Thats why she goes to zoo to see that."
question = generate_question(text, modefor x in context[0]l, tokenizer)
prifor x in context[0]nt("Generated Question:", question)


Generated Question: Why does Mahima go to the zoo?


In [71]:
actual_questions=[x for x in val_df['question'][:2000]]
actual_questions
# len(actual_questions)

['To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'What is in front of the Notre Dame Main Building?',
 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?',
 'What is the Grotto at Notre Dame?',
 'What sits on top of the Main Building at Notre Dame?',
 'When did the Scholastic Magazine of Notre dame begin publishing?',
 "How often is Notre Dame's the Juggler published?",
 'What is the daily student paper at Notre Dame called?',
 'How many student news papers are found at Notre Dame?',
 'In what year did the student paper Common Sense begin publication at Notre Dame?',
 'Where is the headquarters of the Congregation of the Holy Cross?',
 'What is the primary seminary of the Congregation of the Holy Cross?',
 'What is the oldest structure at Notre Dame?',
 'What individuals live at Fatima House at Notre Dame?',
 'Which prize did Frederick Buechner create?',
 'How many BS level degrees are offered in the College of Engineering at Notre

In [70]:
generated_question=[ generate_question(x) for x in context[:2000]]
generated_question

['What is a replica of the grotto at Lourdes?',
 'What is a replica of the grotto at Lourdes?',
 'What is a replica of the grotto at Lourdes?',
 'What is a replica of the grotto at Lourdes?',
 'What is a replica of the grotto at Lourdes?',
 'When was the Scholastic magazine first published?',
 'When was the Scholastic magazine first published?',
 'When was the Scholastic magazine first published?',
 'When was the Scholastic magazine first published?',
 'When was the Scholastic magazine first published?',
 'What is the main seminary of Notre Dame?',
 'What is the main seminary of Notre Dame?',
 'What is the main seminary of Notre Dame?',
 'What is the main seminary of Notre Dame?',
 'What is the main seminary of Notre Dame?',
 'In what year was the College of Engineering established?',
 'In what year was the College of Engineering established?',
 'In what year was the College of Engineering established?',
 'In what year was the College of Engineering established?',
 'In what year was th

In [72]:
from nltk.translate.bleu_score import corpus_bleu
from nltk import word_tokenize

tokenized_generated = [word_tokenize(q.lower()) for q in generated_question]
tokenized_actual = [[word_tokenize(q.lower()) for q in refs] for refs in actual_questions]


bleu_score = corpus_bleu(tokenized_actual, tokenized_generated)
print("BLEU Score:", bleu_score)


BLEU Score: 0.5724251831963729
